# RegVelo benchmark on cell cycle data

Notebook benchmarks velocity, latent time and GRN inference, and cross boundary correctness using RegVelo on cell cycle data.

## Library imports

In [12]:
import numpy as np
import pandas as pd
import torch

import anndata as ad
import scvi
import scanpy as sc

import scvelo as scv
from cellrank.kernels import VelocityKernel
from regvelo import REGVELOVI

from rgv_tools import DATA_DIR
from rgv_tools.benchmarking import get_grn_auroc_cc, get_time_correlation, set_output

## General settings

In [2]:
scv.settings.verbosity = 3

## Constants

In [3]:
DATASET = "cell_cycle"

In [4]:
STATE_TRANSITIONS = [("G1", "S"), ("S", "G2M")]

In [5]:
SAVE_DATA = True
if SAVE_DATA:
    (DATA_DIR / "results").mkdir(parents=True, exist_ok=True)

In [6]:
nn_levels = [10,30,50,70,90,100]

## Functions

In [7]:
def compute_confidence(adata, vkey="velocity"):
    velo = adata.layers[vkey]
    scv.tl.velocity_graph(adata, vkey=vkey, n_jobs=1)
    scv.tl.velocity_confidence(adata, vkey=vkey)

    g_df = pd.DataFrame()
    g_df[f"{vkey}_consistency"] = adata.obs[f"{vkey}_confidence"].to_numpy().ravel()

    return g_df

## Velocity pipeline

In [13]:
for level in nn_levels:
    adata = ad.io.read_h5ad(DATA_DIR / "processed" / f"adata_processed_nn{level}.h5ad")
    W = torch.ones((adata.n_vars, adata.n_vars), dtype=int)
    sc.pp.neighbors(adata, n_neighbors=30, n_pcs=30) # redefine neighborhood graph to ensure graph is the same when calculate velocity consistency
    
    print(adata)
    scvi.settings.seed = 0
    REGVELOVI.setup_anndata(adata, spliced_layer="Ms", unspliced_layer="Mu")
    vae = REGVELOVI(adata,W = W)
    vae.train()
    
    set_output(adata, vae, n_samples=30, batch_size=adata.n_obs)

    t_df = compute_confidence(adata, vkey="fit_t")
    t_df["Dataset"] = "Cell cycle"
    t_df["Method"] = "regvelo"
    
    v_df = compute_confidence(adata, vkey="velocity")
    v_df["Dataset"] = "Cell cycle"
    v_df["Method"] = "regvelo"
    
    t_df.to_parquet(
        path=DATA_DIR / "results" / f"regvelo_confidence_time_{level}.parquet"
    )
    
    v_df.to_parquet(
        path=DATA_DIR / "results" / f"regvelo_confidence_velocity_{level}.parquet"
    )

2025-09-29 18:18:45.608505: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759162726.401815 2657423 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759162726.782448 2657423 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
[rank: 0] Seed set to 0


AnnData object with n_obs × n_vars = 1146 × 354
    obs: 'phase', 'fucci_time', 'initial_size_unspliced', 'initial_size_spliced', 'initial_size', 'n_counts'
    var: 'ensum_id', 'gene_count_corr', 'means', 'dispersions', 'dispersions_norm', 'highly_variable', 'velocity_gamma', 'velocity_qreg_ratio', 'velocity_r2', 'velocity_genes'
    uns: 'log1p', 'neighbors', 'pca', 'velocity_params'
    obsm: 'X_pca'
    varm: 'PCs'
    layers: 'Ms', 'Mu', 'spliced', 'total', 'unspliced', 'velocity'
    obsp: 'connectivities', 'distances'


/home/icb/weixu.wang/miniconda3/envs/regvelo_test/lib/python3.10/site-packages/lightning/fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/icb/weixu.wang/miniconda3/envs/regvelo_test/li ...
/home/icb/weixu.wang/miniconda3/envs/regvelo_test/lib/python3.10/site-packages/lightning/fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/icb/weixu.wang/miniconda3/envs/regvelo_test/li ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/icb/weixu.wang/miniconda3/envs/regvelo_test/lib/python3.10/site-packages/lightning/fabric/plugins/environments/slurm.py:204: The `srun` command 

Training:   0%|          | 0/1500 [00:00<?, ?it/s]

Monitored metric elbo_validation did not improve in the last 45 records. Best score: -882.015. Signaling Trainer to stop.
computing velocity graph (using 1/128 cores)


...

Monitored metric elbo_validation did not improve in the last 45 records. Best score: -1173.876. Signaling Trainer to stop.
computing velocity graph (using 1/128 cores)


  0%|          | 0/1146 [00:00<?, ?cells/s]

    finished (0:00:02) --> added 
    'fit_t_graph', sparse matrix with cosine correlations (adata.uns)
--> added 'fit_t_length' (adata.obs)
--> added 'fit_t_confidence' (adata.obs)
--> added 'fit_t_confidence_transition' (adata.obs)
computing velocity graph (using 1/128 cores)


  0%|          | 0/1146 [00:00<?, ?cells/s]

    finished (0:00:01) --> added 
    'velocity_graph', sparse matrix with cosine correlations (adata.uns)
--> added 'velocity_length' (adata.obs)
--> added 'velocity_confidence' (adata.obs)
--> added 'velocity_confidence_transition' (adata.obs)
